## The covariance matrix completion problem provided by cvxpy tutorial

In [1]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random SDP.
n = 3
p = 3
np.random.seed(1)
C = np.random.randn(n, n)
A = []
b = []
for i in range(p):
    A.append(np.random.randn(n, n))
    b.append(np.random.randn())

# Define and solve the CVXPY problem.
# Create a symmetric matrix variable.
X = cp.Variable((n,n), symmetric=True)
# The operator >> denotes matrix inequality.
constraints = [X >> 0]
constraints += [
    cp.trace(A[i] @ X) == b[i] for i in range(p)
]
prob = cp.Problem(cp.Minimize(cp.trace(C @ X)),
                  constraints)
prob.solve()

# Print result.
print("The optimal value is", prob.value)
print("A solution X is")
print(X.value)

The optimal value is 2.654351510231532
A solution X is
[[ 1.60805795 -0.59770492 -0.69576152]
 [-0.59770492  0.22229041  0.24689363]
 [-0.69576152  0.24689363  1.39679885]]


## Use SDP to solve the matrix conditioning problem

The matrix conditioning problem tries to minimize the difference between the largest and smallest element of a matrix $S$, while keeping $S$ as a linear combination of data matrices $B,~A_i$:

$$
\begin{aligned}
\min_{w,S} &~\lambda_{max}(S)-\lambda_{min}(S)\\
s.t.& ~S = B - \sum_{i=1}^n w_i A_i
\end{aligned}
$$

Writing this in standard SDP form, it is

$$
\begin{aligned}
\min_{w,S} &~\mu-\lambda\\
s.t.&\\
\mu I&\succeq S\succeq \lambda I\\
S &= B - \sum_{i=1}^n w_i A_i
\end{aligned}
$$

In [2]:
# Data
B=np.array([[1,3],[3,1]])
As = np.vstack([np.random.rand(2,2) for _ in range(3)])

# Variables
mu = cp.Variable()
l = cp.Variable()
w = cp.Variable(3)
S = cp.Variable((2,2),symmetric=True)

constraints = [S==B-cp.hstack([w*np.eye(2) for w in w]) @ As,S>>l*np.eye(2),S<<mu*np.eye(2),mu>=0,l>=0]
prob = cp.Problem(cp.Minimize(mu-l),constraints)

In [3]:
prob.solve()
print(prob.value)
print(mu.value,l.value,w.value,S.value)

inf
None None None None


In [4]:
np.linalg.eigvals(S.value)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

## FIM optimization in a local range

In [37]:
# Import packages.
import cvxpy as cp
from utils.dLdp import dhdr,d2hdr2,analytic_dhdz,analytic_FIM
from utils.DynamicFilters import joint_meas_func

def trust_region_plan_single_step(q,ps,C1s,C0s,ks,bs,bounding_radius = 1):
    '''
    bounding_radius: The physical distance reachable in one time step by the sensors' ability.
    '''
    # Two important helper functions for the trust region
    def trust_region_adjust(prob,step_size,new_p,new_z,new_Fp,p_iter,z_iter,Fp_iter):
        feasible = np.all([c.value for c in prob.constraints])
        improved = np.min(np.linalg.eigvals(new_Fp))>np.min(np.linalg.eigvals(Fp_iter))
        if feasible and improved: # To be implemented later
            new_step_size = step_size*2
            return new_p,new_z,new_step_size
        else:
            new_step_size = step_size/2
            return p_iter,z_iter,new_step_size
    
    def solve_linearized_problem(step_size,bounding_radius,p_init,p_iter,z_iter,C1s,C0s,ks,bs):
        # Prepare the raw data
        rs = np.linalg.norm(p_iter-q,axis=1)
        r_hat = ((p_iter-q).T/rs).T
        t_hat=np.zeros(r_hat.shape)
        t_hat[:,0]=-r_hat[:,1]
        t_hat[:,1]=r_hat[:,0]

        d = dhdr(rs,C1s,C0s,ks,bs)
        dd = d2hdr2(rs,C1s,C0s,ks,bs)       

        As = (-d*r_hat.T).T

        Fp=As.T.dot(As) # Current FIM

        # Declare the optimization-related variables.
        dps = cp.Variable(p_iter.shape)
        dz = cp.Variable(nonneg=True)

        # Prepare the matrix directional derivative DF[dps] matrix.
        rhat_conj=[r_hat[i,:,np.newaxis].dot(r_hat[i,np.newaxis,:]) for i in range(len(r_hat))]
        that_conj = [t_hat[i,:,np.newaxis].dot(t_hat[i,np.newaxis,:]) for i in range(len(t_hat))]
        dpdAs =[-dd*rc-1/r*d*tc for dd,d,r,rc,tc in zip(dd,d,rs,rhat_conj,that_conj)]

        dps_As = [dps.T[:,i,np.newaxis]@(As[i,np.newaxis,:]) for i in range(len(As))]

        half_DF = np.hstack(dpdAs)@(cp.vstack(dps_As))

        DF = half_DF+half_DF.T

        # Prepare the objective and constraints
        I = np.eye(p_iter.shape[1])
        constraints = [Fp-z_iter*I+DF-dz*I>>0,
                       cp.norm(p_iter+dps-p_init,axis=1)<=bounding_radius,
                       cp.norm(dps,axis=1)<=step_size]
        prob = cp.Problem(cp.Minimize(-dz),constraints)
        prob.solve()
        newFp = analytic_FIM(q,p_iter+dps.value,C1s,C0s,ks,bs)
        return p_iter+dps.value,newFp,z_iter+prob.value,prob

      
    # The initial z should take value zero.
    z_iter = 0
    # The initial p should take the value of current ps
    p_init = ps
    p_iter = p_init

    # The initial step size is 0.1
    step_size = 0.1

    # Enter the inner loop of solving a sequence of linearized SDP
    for i in range(100):
        Fp_iter = analytic_FIM(q,p_iter,C1s,C0s,ks,bs)
        new_p,new_Fp,new_z,prob = solve_linearized_problem(step_size,bounding_radius,p_init,p_iter,z_iter,C1s,C0s,ks,bs)
        p_iter,z_iter,step_size=trust_region_adjust(prob,step_size,new_p,new_z,new_Fp,p_iter,z_iter,Fp_iter) # Adjust the iteration parameters
        if step_size<1e-4: # Optimality certificate
            break
            
    return p_iter
  
src_locs = np.array([[8.,8.]])
r=0.5
thetas = np.pi * 2 * np.linspace(0,1,3,endpoint=False)
sensor_locs = r*np.vstack([np.sin(thetas),np.cos(thetas)]).T+np.array([[0.,0]])


N_iter=10

# Set up mobile sensors and sources
sensor_names = ["mobile_sensor_{}".format(i+1) for i in range(len(sensor_locs))]
mobile_sensors = [Robot(loc,name) for loc,name in zip(sensor_locs,sensor_names)]

src_names = ["source_{}".format(i+1) for i in range(len(src_locs))]
source = [Robot(loc,name) for loc,name in zip(src_locs,src_names)]

# Set up virtual sensors
C1=0
C0=0
k=1
b=-2
std = 0.1


n_sen = len(mobile_sensors)
n_src = len(source)
C1s=C1*np.ones(n_sen)
C0s = C0*np.ones(n_sen)
ks = k * np.ones(n_sen)
bs = b*np.ones(n_sen)  


h=partial(joint_meas_func,C1s,C0s,ks,bs)# Freeze the coefficients, the signature becomes h(z,ps)
dhdz=partial(analytic_dhdz,C1s,C0s,ks,bs)
# Enter main loop

# Measure
ps = np.array([r.loc for r in mobile_sensors])
q = np.array([s.loc for s in source])


# Plan
